In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report
from tqdm import tqdm

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_excel("/data2/users/ssultana/dataset/N3C_benchmarking/N3C_full_data.xlsx")

In [3]:
df.drop(columns=[1, '1.1'], inplace=True)

In [4]:
df

,"78473, 80767, 764599, 4062484, 4149382, 4169954, 4203722, 4223659, 4251903, 4306780+31.0+0+1+0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","23325, 25844, 30437, 77670, 197381, 198263, 200219, 201061, 315078, 437833, 442077, 4000609, 4209423, 4306292, 40481385, 40482267+32.0+0+1+808+38003564.0+8532.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+1.0+0.0","195083, 315078, 4152351+38.0+0+1+0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","201826, 254761, 257012, 261687, 320128, 438720, 4166902, 4273307, 37016349+64.0+0+1+0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","137989, 138717, 255573, 257004, 312437, 313217, 315282, 320128, 433316, 764123, 4154290, 4322024, 42538119+65.0+0+0+601+38003564.0+8532.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","24134, 372324, 378253, 762296, 4038038, 4166125, 4273307, 4288310+71.0+0+0+460+38003564.0+8532.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","435950, 436826, 437116, 4115367, 4164959, 40305987+60.0+0+1+801+38003563.0+8532.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","0, 132412, 132797, 134116, 138384, 139902, 140673, 194133, 197320, 318800, 320128, 377844, 432250, 432791, 433736, 436096, 437191, 437474, 437663, 438485, 439221, 439224, 440276, 440615, 441267, 441488, 443272, 4060985, 4064036, 4117957, 4144111, 4156229, 4169580, 4174262, 4186463, 4214956, 4226022, 4237320, 4240903, 4241527, 4305831, 4314870, 4322175, 4345578, 36712821, 36712860, 37016200, 43021283, 46272450+50.0+0+1+652+38003564.0+8532.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","75036, 77074, 77965, 78508, 79908, 81175, 81878, 192963, 198678, 198803, 255573, 256451, 257011, 313217, 317002, 320128, 372448, 432867, 436230, 438720, 439297, 439777, 440029, 764123, 4057826, 4110914, 4144111, 4153380, 4154290, 4223659, 4261842, 4330445, 40479625, 40481919, 42537730, 46271022+88.0+0+1+820+38003564.0+8507.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","27674, 138825, 194133, 196360, 198401, 200174, 200843, 201337, 201620, 201826, 254761, 260123, 320128, 374367, 374375, 377889, 379805, 432867, 436070, 437246, 438699, 439777, 442019, 443211, 443597, 444131, 762289, 4031106, 4032787, 4041285, 4079750, 4084229, 4092743, 4110815, 4115991, 4155909, 4170554, 4171917, 4185946, 4193704, 4223659, 4236484, 4271013, 43531578, 45757656+86.0+0+0+212+38003564.0+8507.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0",...,"320128, 320536, 4223659, 4273307+57.0+0+1+460+38003564.0+8507.0+8522.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","195867, 434005, 436230, 4092538, 4193704, 4282096, 40481920+29.0+0+1+0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","81239, 133810, 192450, 193782, 194686, 200174, 201826, 254761, 255848, 257004, 312437, 316139, 319835, 320136, 320536, 376065, 432867, 432870, 435515, 436230, 437113, 437663, 439696, 439777, 442019, 442077, 442111, 443729, 443919, 444101, 444459, 764123, 4007310, 4093531, 4162994, 4180628, 4187218, 4209423, 4216644, 4273307, 4310235, 4320635, 37016114, 37017432, 37309626, 40479576, 40481043, 40483287, 42537748, 43530690, 43531578, 44782429, 44784621, 45768812+60.0+0+0+474+38003564.0+8507.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","77234, 78232, 135618, 198803, 255848, 374009, 377091, 435783, 435796, 436070, 436096, 437113, 437663, 439777, 442077, 443782, 444070, 4133224, 4144111, 4170554, 4224940, 4254485, 4273307, 4286201, 4336254+63.0+0+1+462+38003564.0+8507.0+8527.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0+0.0","27674, 31967, 81902, 139900, 141371, 194081, 197684, 254761, 314754, 378253, 432545, 438028, 439235, 440383, 442077, 4002818, 4041283, 4077577, 4092411, 4152376, 4177206, 4196024, 4224968, 4282096, 4300092+23.0+0+1+

In [5]:
np_arr = df.values
np_arr1 = np.reshape(np_arr, (-1))
df = pd.DataFrame(np_arr1)

In [6]:
df

,0
0,"140214, 254761, 257011, 377889, 4141481+6.0+0+..."
1,"78473, 137548, 4236484, 37311338, 37311341+41...."
2,"195867, 197684, 201078, 4031128, 4289526, 3701..."
3,"133834, 140214, 438134, 438398, 442588, 444131..."
4,"25297, 254761+38.0+0+1+0+0.0+0.0+0.0+0.0+0.0+0..."
...,...
442995,"25297, 28060, 77670, 78272, 80552, 140480, 141..."
442996,"74722, 75910, 77670, 78232, 80502, 132344, 133..."
442997,"24134, 75909, 77074, 77670, 78232, 80502, 8187..."
442998,"140673, 373785, 378425, 4144111, 4265749, 3710..."


In [7]:
df[['conditions', 'age' ,'severity_covid_death', 'outcome', 'zip', 'ethnicity_concept_id', 'gender_concept_id', 'race_concept_id','trazodone', 'amitriptyline', 'fluoxetine', 'citalopram', 'paroxetine', 'venlafaxine', 'vilazodone', 'vortioxetine', 'sertraline', 'bupropion', 'mirtazapine', 'desvenlafaxine', 'doxepin', 'duloxetine', 'escitalopram', 'nortriptyline']] = df[0].apply(lambda x: pd.Series(str(x).split('+')))
df.drop(columns=0, inplace=True)

In [8]:
df

,conditions,age,severity_covid_death,outcome,zip,ethnicity_concept_id,gender_concept_id,race_concept_id,trazodone,amitriptyline,...,vilazodone,vortioxetine,sertraline,bupropion,mirtazapine,desvenlafaxine,doxepin,duloxetine,escitalopram,nortriptyline
0,"140214, 254761, 257011, 377889, 4141481",6.0,0,1,631,38003564.0,8532.0,8516.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"78473, 137548, 4236484, 37311338, 37311341",41.0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"195867, 197684, 201078, 4031128, 4289526, 3701...",62.0,0,1,970,38003563.0,8532.0,8527.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"133834, 140214, 438134, 438398, 442588, 444131...",26.0,0,1,463,0.0,8507.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"25297, 254761",38.0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442995,"25297, 28060, 77670, 78272, 80552, 140480, 141...",16.0,0,0,841,38003563.0,8532.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
442996,"74722, 75910, 77670, 78232, 80502, 132344, 133...",75.0,0,0,145,38003564.0,8507.0,8516.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442997,"24134, 75909, 77074, 77670, 78232, 80502, 8187...",79.0,0,1,605,38003564.0,8532.0,8527.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442998,"140673, 373785, 378425, 4144111, 4265749, 3710...",45.0,0,1,524,38003564.0,8532.0,8527.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df.conditions = df.conditions.apply(lambda x: str(x).split(','))
df.conditions = df.conditions.apply(lambda x: list(map(int, x)))

In [10]:
df

,conditions,age,severity_covid_death,outcome,zip,ethnicity_concept_id,gender_concept_id,race_concept_id,trazodone,amitriptyline,...,vilazodone,vortioxetine,sertraline,bupropion,mirtazapine,desvenlafaxine,doxepin,duloxetine,escitalopram,nortriptyline
0,"[140214, 254761, 257011, 377889, 4141481]",6.0,0,1,631,38003564.0,8532.0,8516.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[78473, 137548, 4236484, 37311338, 37311341]",41.0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[195867, 197684, 201078, 4031128, 4289526, 370...",62.0,0,1,970,38003563.0,8532.0,8527.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[133834, 140214, 438134, 438398, 442588, 44413...",26.0,0,1,463,0.0,8507.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[25297, 254761]",38.0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442995,"[25297, 28060, 77670, 78272, 80552, 140480, 14...",16.0,0,0,841,38003563.0,8532.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
442996,"[74722, 75910, 77670, 78232, 80502, 132344, 13...",75.0,0,0,145,38003564.0,8507.0,8516.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442997,"[24134, 75909, 77074, 77670, 78232, 80502, 818...",79.0,0,1,605,38003564.0,8532.0,8527.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
442998,"[140673, 373785, 378425, 4144111, 4265749, 371...",45.0,0,1,524,38003564.0,8532.0,8527.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def convert_to_int(x):
    try:
        return int(float(x))
    except ValueError:
        return 0
for col in df.columns[1:]:
    df.loc[:,col] = df.loc[:,col].apply(convert_to_int)

In [12]:
df.drop(columns=['severity_covid_death', 'zip', 'ethnicity_concept_id', 'gender_concept_id', 'race_concept_id', 'trazodone', 'amitriptyline', 'fluoxetine', 'citalopram', 'paroxetine', 'venlafaxine', 'vilazodone', 'vortioxetine', 'sertraline', 'bupropion', 'mirtazapine', 'desvenlafaxine', 'doxepin', 'duloxetine', 'escitalopram', 'nortriptyline'], inplace=True)
df

,conditions,age,outcome
0,"[140214, 254761, 257011, 377889, 4141481]",6,1
1,"[78473, 137548, 4236484, 37311338, 37311341]",41,1
2,"[195867, 197684, 201078, 4031128, 4289526, 370...",62,1
3,"[133834, 140214, 438134, 438398, 442588, 44413...",26,1
4,"[25297, 254761]",38,1
...,...,...,...
442995,"[25297, 28060, 77670, 78272, 80552, 140480, 14...",16,0
442996,"[74722, 75910, 77670, 78232, 80502, 132344, 13...",75,0
442997,"[24134, 75909, 77074, 77670, 78232, 80502, 818...",79,1
442998,"[140673, 373785, 378425, 4144111, 4265749, 371...",45,1


In [13]:
# Loading concept id to snomed mapping.

df_sn2con = pd.read_excel("/data2/users/ssultana/dataset/N3C_benchmarking/snomed_to_concept_id.xlsx")
df_sn2con.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
np_arr = df_sn2con.values
np_arr1 = np.reshape(np_arr, (-1))
df_sn2con = pd.DataFrame(np_arr1[:-30])

In [15]:
df_sn2con[['snomed_id', 'condition_concept_id']] = df_sn2con[0].apply(lambda x: pd.Series(str(x).split('+')))
df_sn2con.drop(columns=0, inplace=True)

df_sn2con.snomed_id = df_sn2con.snomed_id.astype(int)
df_sn2con.condition_concept_id = df_sn2con.condition_concept_id.apply(lambda x: int(float(x)))

In [16]:
df_sn2con_dict = dict(zip(df_sn2con.condition_concept_id, df_sn2con.snomed_id))
df['snomed_conditions'] = df.conditions.apply(lambda con: [df_sn2con_dict[x] for x in con])

In [17]:
df.drop(columns=['conditions'], inplace=True)

In [18]:
df

,age,outcome,snomed_conditions
0,6,1,"[271807003, 49727002, 54398005, 15188001, 2661..."
1,41,1,"[266578003, 54404000, 91019004, 816055008, 816..."
2,62,1,"[22741003, 49650001, 52441000, 109351006, 3696..."
3,26,1,"[24079001, 271807003, 77692006, 111583006, 782..."
4,38,1,"[363746003, 49727002]"
...,...,...,...
442995,16,0,"[363746003, 43878008, 29857009, 70704007, 4281..."
442996,75,0,"[25343008, 2089002, 29857009, 267949000, 64859..."
442997,79,1,"[81680005, 76069003, 57676002, 29857009, 26794..."
442998,45,1,"[40930008, 10810001, 41446000, 266435005, 3975..."


In [19]:
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer()

# df = pd.DataFrame(mlb.fit_transform(df['snomed_conditions']),columns=mlb.classes_, index=df.index)
# print (df)

In [20]:
with open('/data1/mrahman/snomed_embeddings/line.json') as f:
    line_emb = json.load(f)
# line_emb[14265004]
line_emb = {int(k):v for k,v in line_emb.items()}

In [21]:
def mean_emb(snmd_con):
    p = np.array([line_emb.get(x,[0]*128) for x in snmd_con])
    p = p[~np.all(p == 0, axis=1)]
    p = p.mean(axis=0)
    return p

In [22]:
embaded_snmd = df.snomed_conditions.apply(mean_emb)
embaded_snmd
embaded_snmd.isna().sum()
embaded_snmd_line_arr = np.array(embaded_snmd.values.tolist())
embaded_snmd_line_arr = np.nan_to_num(embaded_snmd_line_arr)

In [23]:
s_id = pd.DataFrame(embaded_snmd_line_arr, columns=list(range(0, 128)))

In [24]:
s_id.reset_index(drop=True, inplace=True)
df['age'].reset_index(drop=True, inplace=True)

In [25]:
X = pd.concat([df[['age']], s_id], axis=1)
Y = df['outcome']

In [26]:
# Encode age as 1, Unpriviledged as 0
X.loc[X.age < 65, 'age'] = 1
X.loc[X.age > 65, 'age'] = 0

In [61]:
X

,age,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,1,0.001320,-0.001358,0.002084,0.003564,-0.003194,-0.002017,0.001431,-0.001622,0.001089,...,0.003053,0.004080,0.000644,0.000411,-0.000884,-0.002012,0.002476,0.000626,-0.002627,-0.001140
1,1,0.000648,-0.002621,0.002994,-0.000147,0.002184,0.002705,-0.000846,-0.001370,0.003216,...,-0.003449,0.000419,0.003201,-0.001343,-0.000485,-0.003039,0.001567,0.001480,-0.002352,0.001427
2,1,0.004674,0.001998,0.002030,0.002025,-0.001051,-0.000277,0.001123,-0.000182,-0.000464,...,-0.006437,0.001706,0.000031,0.005099,0.002060,-0.000326,-0.002678,0.002073,0.001232,-0.001729
3,1,0.001593,-0.000915,-0.000376,0.002417,-0.001285,-0.002320,0.000633,0.000135,0.000067,...,0.004135,-0.002713,0.002329,-0.001288,0.001073,0.001029,0.001538,-0.000912,0.002028,-0.000900
4,1,0.003684,-0.006422,0.003219,-0.002425,-0.002449,-0.000478,0.003236,-0.004667,0.000084,...,-0.001524,0.002558,-0.000471,0.001785,0.003185,-0.001324,-0.004981,-0.000333,0.000254,-0.002200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442995,1,-0.000467,-0.000825,0.000370,-0.000290,-0.001395,-0.000198,0.001334,0.000422,-0.001215,...,0.000451,0.000471,-0.001475,0.000336,-0.000064,0.001415,-0.000750,0.001002,-0.000078,0.000674
442996,0,-0.000658,-0.000474,0.000412,0.000091,0.000035,-0.000488,-0.000074,-0.001617,-0.000129,...,0.000510,0.000491,-0.001093,0.000509,0.000042,0.000419,-0.000706,-0.001100,0.000032,0.000349
442997,0,0.000509,-0.001067,0.000397,0.000728,-0.000040,0.000596,-0.000341,-0.000141,-0.000070,...,0.001042,0.001216,0.001001,-0.001181,0.000718,-0.000623,0.001059,-0.000044,0.000049,0.000059
442998,1,0.001082,-0.000088,0.001382,0.000173,-0.001729,-0.001714,0.001024,-0.000282,0.001644,...,-0.000406,-0.000125,0.001609,-0.000257,0.001249,0.000658,0.001954,-0.000659,0.002196,-0.001233


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Biased Model

## 1. Logistic Regression

In [28]:
#logistic regression

biased_lr = LogisticRegression(max_iter=1000 ,C=1, verbose=1, n_jobs=30).fit(X_train,y_train)

y_lr_pred=biased_lr.predict(X_test)
acc_lr_biased = metrics.accuracy_score(y_test, y_lr_pred)
print("Accuracy of Logistic Regression (Biased Dataset): {:0.3f}".format(acc_lr_biased))
# cr_lr_biased = classification_report(y_test, y_lr_pred)
# print("AClassification Report of Logistic Regression (Biased Dataset):\n", cr_lr_biased)

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 out of   1 | elapsed:  4.0min finished


Accuracy of Logistic Regression (Biased Dataset): 0.704


## 2. Random Forest 

In [29]:
biased_rf=RandomForestClassifier(n_estimators=100, verbose=1, n_jobs=30)

biased_rf.fit(X_train,y_train)

y_rf_pred=biased_rf.predict(X_test)
acc_rf_biased = metrics.accuracy_score(y_test, y_rf_pred)
print("Accuracy of Random Forest (Biased Dataset): {:0.3f}".format(acc_rf_biased))

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:   46.8s finished
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.


Accuracy of Random Forest (Biased Dataset): 0.759


[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:    0.4s finished


## 3. Support Vector Machine (SVM)

In [30]:
#SVM
biased_svm= svm.SVC(kernel='linear', verbose=1, max_iter=10000)
biased_svm.fit(X_train,y_train)

y_svm_pred=biased_svm.predict(X_test)


[LibSVM]..

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          130     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.45651D+05    |proj g|=  1.04611D+05

At iterate   50    f=  2.12701D+05    |proj g|=  3.16118D+03

At iterate  100    f=  2.12584D+05    |proj g|=  3.30809D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  130    114    132      1     0     0   1.242D+01   2.126D+05
  F =   212580.12057693905     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
.......WARN: libsvm Solver reached max_iter
optimization finished, #it

In [31]:
acc_svm_biased = metrics.accuracy_score(y_test, y_svm_pred)
print("Accuracy of SVM (Biased Dataset): {:0.3f}".format(acc_svm_biased))

Accuracy of SVM (Biased Dataset): 0.704


## 4. Neural Network

In [32]:
#Neural Network

model = Sequential()
model.add(Dense(12, input_dim=129, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=16, batch_size=1000)
model.evaluate(X_test, y_test)
y_nn_pred=model.predict(X_test).round()

acc_nn_biased = metrics.accuracy_score(y_test, y_nn_pred)
print("Accuracy of Neural Network (Biased Dataset): {:0.3f}".format(acc_nn_biased))
precision_nn_biased = metrics.precision_score(y_test, y_nn_pred)
print("Precision of Neural Network (Biased Dataset): {:0.3f}".format(precision_nn_biased))
recall_nn_biased = metrics.recall_score(y_test, y_nn_pred)
print("Recall of Neural Network (Biased Dataset): {:0.3f}".format(recall_nn_biased))

2022-11-03 12:10:07.824849: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-03 12:10:07.824950: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thor): /proc/driver/nvidia/version does not exist


Epoch 1/16
355/355 [==============================] - 4s 6ms/step - loss: 0.6022 - accuracy: 0.7080 - val_loss: 0.5673 - val_accuracy: 0.7229
Epoch 2/16
355/355 [==============================] - 2s 5ms/step - loss: 0.5651 - accuracy: 0.7250 - val_loss: 0.5654 - val_accuracy: 0.7199
Epoch 3/16
355/355 [==============================] - 2s 5ms/step - loss: 0.5624 - accuracy: 0.7252 - val_loss: 0.5613 - val_accuracy: 0.7232
Epoch 4/16
355/355 [==============================] - 2s 5ms/step - loss: 0.5596 - accuracy: 0.7260 - val_loss: 0.5587 - val_accuracy: 0.7242
Epoch 5/16
355/355 [==============================] - 2s 5ms/step - loss: 0.5573 - accuracy: 0.7271 - val_loss: 0.5567 - val_accuracy: 0.7249
Epoch 6/16
355/355 [==============================] - 2s 5ms/step - loss: 0.5557 - accuracy: 0.7284 - val_loss: 0.5555 - val_accuracy: 0.7238
Epoch 7/16
355/355 [==============================] - 2s 5ms/step - loss: 0.5547 - accuracy: 0.7290 - val_loss: 0.5544 - val_accuracy: 0.7255
Epoch 

# AIF360

## Import necessary libraries

In [33]:
from IPython.display import display, Markdown
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
from sklearn.preprocessing import StandardScaler


# IBM's fairness tooolbox:
from aif360.datasets import BinaryLabelDataset  # To handle the data
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric  # For calculating metrics
from aif360.explainers import MetricTextExplainer  # For explaining metrics

# Preprocessing technique
from aif360.algorithms.preprocessing import Reweighing  # Preprocessing technique
from aif360.algorithms.preprocessing import DisparateImpactRemover # Preprocessing technique
from aif360.algorithms.preprocessing.lfr import LFR # Preprocessing technique
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_german

# Inprocessing technique
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing.exponentiated_gradient_reduction import ExponentiatedGradientReduction

# Post-processing algorithm
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification

In [34]:
train_pp_bld = BinaryLabelDataset(df=pd.concat((X_train, y_train),
                                               axis=1),
                                  label_names=['outcome'],
                                  protected_attribute_names=['age'],
                                  favorable_label=1,
                                  unfavorable_label=0)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [35]:
train_pp_bld

               instance weights            features                          \
                                protected attribute                           
                                                age             0         1   
instance names                                                                
300540                      1.0                 1.0  6.602000e-04 -0.000992   
37374                       1.0                 1.0  4.039167e-04 -0.000230   
144904                      1.0                 1.0 -2.627500e-03 -0.000679   
244352                      1.0                 1.0  0.000000e+00  0.000000   
421368                      1.0                 1.0 -9.230769e-07 -0.000072   
...                         ...                 ...           ...       ...   
32549                       1.0                 1.0 -1.419000e-03  0.002717   
423282                      1.0                 1.0  2.582500e-03 -0.001489   
200646                      1.0                 1.0 

In [36]:
test_pp_bld = BinaryLabelDataset(df=pd.concat((X_test, y_test),
                                               axis=1),
                                  label_names=['outcome'],
                                  protected_attribute_names=['age'],
                                  favorable_label=1,
                                  unfavorable_label=0)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

## Check the original dataset is biased or not

In [37]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(train_pp_bld, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.statistical_parity_difference())
print("Train set: Compute Disparate Impact ratio for unprivileged and privileged groups = %f" % metric_orig_train.disparate_impact())
print("Train set: Compute base rate for privileged groups = %f" % metric_orig_train.base_rate(privileged=privileged_groups))



#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.327738
Train set: Compute Disparate Impact ratio for unprivileged and privileged groups = 0.576708
Train set: Compute base rate for privileged groups = 0.774261


In [38]:
# Metric for the original dataset
metric_orig_test = BinaryLabelDatasetMetric(test_pp_bld, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())
print("Test set: Compute Disparate Impact ratio for unprivileged and privileged groups = %f" % metric_orig_test.disparate_impact())
print("Test set: Compute base rate for privileged groups = %f" % metric_orig_test.base_rate(privileged=privileged_groups))

#### Original test dataset

Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.317301
Test set: Compute Disparate Impact ratio for unprivileged and privileged groups = 0.589934
Test set: Compute base rate for privileged groups = 0.773780


Comment 1: Mean difference of training and test dataset shows that privileged group is getting almost 32% more fabourable outcomes in the training dataset. So bias exists that needs to reduce.

# Pre-processing Bias Mitigation Algorithms: applied to the training data and modifies the training samples to reduce biasness.

1. Reweighing pre-processing
2. Disparate Impact Ratio pre-processing
3. Learning Fair Representation pre-processing
4. Optimized pre-processing

## 1. Rewighing Pre-processing Algorithm

In [39]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
train_pp_bld_rw = RW.fit_transform(train_pp_bld)

In [40]:
### Compute fairness metric on transformed dataset-rw
metric_transf_train_rw = BinaryLabelDatasetMetric(train_pp_bld_rw, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset - Reweighing"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train_rw.mean_difference())
print("Difference Disparate Impact between unprivileged and privileged groups = %f" % metric_transf_train_rw.disparate_impact())

#### Transformed training dataset - Reweighing

Difference in mean outcomes between unprivileged and privileged groups = -0.000000
Difference Disparate Impact between unprivileged and privileged groups = 1.000000


Now the training dataset is unbiased

## Apply classifiers on transformed dataset via Reweighing

In [41]:
scale_transf_rw = StandardScaler()

## transformed training dataset
X_train_rw = scale_transf_rw.fit_transform(train_pp_bld_rw.features)
y_train_rw = train_pp_bld_rw.labels.ravel()

## transformed test dataset
dataset_transf_test_pred_rw = test_pp_bld.copy(deepcopy=True)
X_test_rw = scale_transf_rw.fit_transform(dataset_transf_test_pred_rw.features)
y_test_rw = dataset_transf_test_pred_rw.labels

### 1.1 Logistics Regresssion on Transformed Data(Bias Mitigated Dataset via Reweighing)

In [42]:
lmod_rw = LogisticRegression(max_iter=10000, n_jobs = 30)
lmod_rw.fit(X_train_rw, y_train_rw,
        sample_weight=train_pp_bld_rw.instance_weights)
y_train_pred_rw = lmod_rw.predict(X_train_rw)

In [43]:
y_lr_pred_rw = lmod_rw.predict(X_test_rw)


dataset_debias_test_lr_rw = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_lr_rw.labels = y_lr_pred_rw.ravel()
display(Markdown("#### Model: LR- debiasing with RW - classification metrics"))
classified_metric_debiasing_test_lr_rw = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_lr_rw,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_lr_rw.accuracy())
TPR_lr_rw = classified_metric_debiasing_test_lr_rw.true_positive_rate()
TNR_lr_rw = classified_metric_debiasing_test_lr_rw.true_negative_rate()
bal_acc_debiasing_test_lr_rw = 0.5*(TPR_lr_rw+TNR_lr_rw)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_lr_rw)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_lr_rw.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_lr_rw.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_lr_rw.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_lr_rw.theil_index())

#### Model: LR- debiasing with RW - classification metrics

Test set: Classification accuracy = 0.708025
Test set: Balanced classification accuracy = 0.525985
Test set: Disparate impact = 0.968844
Test set: Equal opportunity difference = -0.008142
Test set: Average odds difference = -0.016777
Test set: Theil_index = 0.076739


### 1.2 Random Forest on Transformed Data(Bias Mitigated Dataset via Reweighing)

In [44]:
rf_rw=RandomForestClassifier(n_estimators=100, verbose=1, n_jobs=30).fit(X_train_rw, y_train_rw,
                                                                            sample_weight=train_pp_bld_rw.instance_weights)
                                                                            
y_rf_pred_rw=rf_rw.predict(X_test_rw)
dataset_debias_test_rf_rw = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_rf_rw.labels = y_rf_pred_rw.ravel()
display(Markdown("#### Model: RF- debiasing with RW - classification metrics"))
classified_metric_debiasing_test_rf_rw = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_rf_rw,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_rf_rw.accuracy())
TPR_rf_rw = classified_metric_debiasing_test_rf_rw.true_positive_rate()
TNR_rf_rw = classified_metric_debiasing_test_rf_rw.true_negative_rate()
bal_acc_debiasing_test_rf_rw = 0.5*(TPR_rf_rw+TNR_rf_rw)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_rf_rw)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_rf_rw.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_rf_rw.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_rf_rw.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_rf_rw.theil_index())

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:   47.4s finished
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:    0.4s finished


#### Model: RF- debiasing with RW - classification metrics

Test set: Classification accuracy = 0.752935
Test set: Balanced classification accuracy = 0.616506
Test set: Disparate impact = 0.743227
Test set: Equal opportunity difference = -0.160190
Test set: Average odds difference = -0.182495
Test set: Theil_index = 0.086854


### 1.3 SVM on Transformed Data(Bias Mitigated Dataset via Reweighing)

In [45]:
#SVM
svm_rw= svm.SVC(kernel='linear', verbose=1, max_iter=10000).fit(X_train_rw, y_train_rw,
                                                                            sample_weight=train_pp_bld_rw.instance_weights)
                                                                            

y_svm_pred_rw=svm_rw.predict(X_test_rw)

[LibSVM].........WARN: libsvm Solver reached max_iter
optimization finished, #iter = 10000
obj = -7532.605080, rho = 0.685099
nSV = 9746, nBSV = 6454
Total nSV = 9746


In [46]:
# acc_svm_rw = metrics.accuracy_score(y_test_rw, y_svm_pred_rw)
# print("Rewighing Preprocessing: SVM accuracy: {:0.3f}".format(acc_svm_rw))
dataset_debias_test_svm_rw = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_svm_rw.labels = y_svm_pred_rw.ravel()
display(Markdown("#### Model: SVM- debiasing with RW - classification metrics"))
classified_metric_debiasing_test_svm_rw = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_svm_rw,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_lr_rw.accuracy())
TPR_svm_rw = classified_metric_debiasing_test_svm_rw.true_positive_rate()
TNR_svm_rw = classified_metric_debiasing_test_svm_rw.true_negative_rate()
bal_acc_debiasing_test_svm_rw = 0.5*(TPR_lr_rw+TNR_lr_rw)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_svm_rw)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_svm_rw.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_svm_rw.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_svm_rw.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_svm_rw.theil_index())

#### Model: SVM- debiasing with RW - classification metrics

Test set: Classification accuracy = 0.708025
Test set: Balanced classification accuracy = 0.525985
Test set: Disparate impact = 1.010823
Test set: Equal opportunity difference = 0.013591
Test set: Average odds difference = 0.009956
Test set: Theil_index = 0.072336


### 1.4 Neural Network on transformed dataset (Reweighing)

In [47]:
#Neural Network

nn_rw = Sequential()
nn_rw.add(Dense(12, input_dim=129, activation='relu'))
nn_rw.add(Dense(8, activation='relu'))
nn_rw.add(Dense(1, activation='sigmoid'))

nn_rw.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_rw.fit(X_train_rw, y_train_rw, validation_data=(X_test_rw, y_test_rw), epochs=16, batch_size=1000)
nn_rw.evaluate(X_test_rw, y_test_rw)
y_nn_pred_rw=nn_rw.predict(X_test_rw).round()

Train on 354400 samples, validate on 88600 samples
Epoch 1/16


2022-11-03 12:55:28.185769: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


354400/354400 [==============================] - 2s 5us/sample - loss: 0.5772 - accuracy: 0.7085 - val_loss: 0.5578 - val_accuracy: 0.7287
Epoch 2/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5528 - accuracy: 0.7336 - val_loss: 0.5446 - val_accuracy: 0.7416
Epoch 3/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5432 - accuracy: 0.7414 - val_loss: 0.5380 - val_accuracy: 0.7469
Epoch 4/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5385 - accuracy: 0.7442 - val_loss: 0.5352 - val_accuracy: 0.7478
Epoch 5/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5359 - accuracy: 0.7459 - val_loss: 0.5336 - val_accuracy: 0.7482
Epoch 6/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5339 - accuracy: 0.7472 - val_loss: 0.5324 - val_accuracy: 0.7493
Epoch 7/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5325 - accuracy: 0.7479 - v

In [48]:
dataset_debias_test_nn_rw = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_nn_rw.labels = y_nn_pred_rw
display(Markdown("#### Model: NN- debiasing with RW - classification metrics"))
classified_metric_debiasing_test_nn_rw = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_nn_rw,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_nn_rw.accuracy())
TPR_nn_rw = classified_metric_debiasing_test_nn_rw.true_positive_rate()
TNR_nn_rw = classified_metric_debiasing_test_nn_rw.true_negative_rate()
bal_acc_debiasing_test_nn_rw = 0.5*(TPR_nn_rw+TNR_nn_rw)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_nn_rw)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_nn_rw.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_nn_rw.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_nn_rw.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_nn_rw.theil_index())

#### Model: NN- debiasing with RW - classification metrics

Test set: Classification accuracy = 0.750847
Test set: Balanced classification accuracy = 0.625044
Test set: Disparate impact = 0.732356
Test set: Equal opportunity difference = -0.166594
Test set: Average odds difference = -0.180465
Test set: Theil_index = 0.098765


In [49]:
# print('Accuracy on Test Set [Preprocessing Algorithm:]:')

# # iterables = [["Logistic Regression", "Random Forest", "SVM", "Neural Network"], ["Classification Accuracy", "Balanced Classification Accuracy", "Disparate Impact", "Equal opportunity difference", "Average odds difference", "Theil Index"]]

# iterables = [["Logistic Regression", "Random Forest", "Neural Network"], ["Classification Accuracy", "Balanced Classification Accuracy", "Disparate Impact", "Equal opportunity difference", "Average odds difference", "Theil Index"]]

# index = pd.MultiIndex.from_product(iterables, names=["Classification", "Fairness Matrics"])



# # df = pd.DataFrame({'Count A': [12., 70., 30., 20.], 'Count B': [12., 70., 30., 20.]}, index=index)


# result_acc = pd.DataFrame({'Classification Accuracy':             [classified_metric_debiasing_test_lr_rw.accuracy(), 
#                                                                   classified_metric_debiasing_test_rf_rw.accuracy(),
#                                                                   classified_metric_debiasing_test_nn_rw.accuracy()
#                                                                   ],
#                         'Balanced Classification Accuracy':       [bal_acc_debiasing_test_lr_rw,
#                                                                   bal_acc_debiasing_test_rf_rw,
#                                                                   bal_acc_debiasing_test_nn_rw
#                                                                   ],
#                         'Disparate impact':                       [classified_metric_debiasing_test_lr_rw.disparate_impact(),
#                                                                   classified_metric_debiasing_test_rf_rw.disparate_impact(),
#                                                                   classified_metric_debiasing_test_nn_rw.disparate_impact()
#                                                                   ],
#                         'Equal opportunity difference':           [classified_metric_debiasing_test_lr_rw.equal_opportunity_difference(),
#                                                                   classified_metric_debiasing_test_rf_rw.equal_opportunity_difference(),
#                                                                   classified_metric_debiasing_test_nn_rw.equal_opportunity_difference()
#                                                                   ],
#                         'Average odds difference':                [classified_metric_debiasing_test_lr_rw.average_odds_difference(),
#                                                                   classified_metric_debiasing_test_rf_rw.average_odds_difference(),
#                                                                   classified_metric_debiasing_test_nn_rw.average_odds_difference()
#                                                                   ],
#                         'Theil_index':                            [classified_metric_debiasing_test_lr_rw.theil_index(),
#                                                                   classified_metric_debiasing_test_rf_rw.theil_index(),
#                                                                   classified_metric_debiasing_test_nn_rw.theil_index()
#                                                                   ],

#                         }, 
#                      index = index)

# result_acc

## 2. Disparate Impact Ratio Pre-processing Algorithm
Disparate Impact Remover is a pre-processing technique that edits values, which will be used as features, to increase fairness between the groups. It compares the proportion of individuals that receive a positive output for two groups: an unprivileged group and a privileged group. The calculation is the proportion of the unprivileged group that received the positive outcome divided by the proportion of the privileged group that received the positive outcome.
The industry standard is a four-fifths rule: if the unprivileged group receives a positive outcome less than 80% of their proportion of the privileged group, this is a disparate impact violation.

In [50]:
DI = DisparateImpactRemover(repair_level=1)
train_pp_bld_di = DI.fit_transform(train_pp_bld)

In [51]:
### Compute fairness metric on transformed dataset-DI
metric_transf_train_di = BinaryLabelDatasetMetric(train_pp_bld_di, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset - Disparate Impact"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train_di.mean_difference())
print("Difference Disparate Impact between unprivileged and privileged groups = %f" % metric_transf_train_di.disparate_impact())

#### Transformed training dataset - Disparate Impact

Difference in mean outcomes between unprivileged and privileged groups = -0.327738
Difference Disparate Impact between unprivileged and privileged groups = 0.576708


## Apply classifiers on transformed dataset via Disparate Impact

In [52]:
scale_transf_di = StandardScaler()

## transformed training dataset
X_train_di = scale_transf_di.fit_transform(train_pp_bld_di.features)
y_train_di = train_pp_bld_di.labels.ravel()

## transformed test dataset
dataset_transf_test_pred_di = test_pp_bld.copy(deepcopy=True)
X_test_di = scale_transf_di.fit_transform(dataset_transf_test_pred_di.features)
y_test_di = dataset_transf_test_pred_di.labels

### 2.1 Logistic Regression on transformed (DI) dataset

In [53]:
lmod_di = LogisticRegression(max_iter=10000, n_jobs = 30, class_weight='balanced', solver='liblinear')
lmod_di.fit(X_train_di, y_train_di,
        sample_weight=train_pp_bld_di.instance_weights)

LogisticRegression(class_weight='balanced', max_iter=10000, n_jobs=30,
                   solver='liblinear')

In [54]:
y_lr_pred_di = lmod_di.predict(X_test_rw)

dataset_debias_test_lr_di = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_lr_di.labels = y_lr_pred_di
display(Markdown("#### Model: LR- debiasing with DI - classification metrics"))
classified_metric_debiasing_test_lr_di = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_lr_di,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_lr_di.accuracy())
TPR_lr_di = classified_metric_debiasing_test_lr_di.true_positive_rate()
TNR_lr_di = classified_metric_debiasing_test_lr_di.true_negative_rate()
bal_acc_debiasing_test_lr_di = 0.5*(TPR_lr_di+TNR_lr_di)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_lr_di)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_lr_di.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_lr_di.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_lr_di.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_lr_di.theil_index())

#### Model: LR- debiasing with DI - classification metrics

Test set: Classification accuracy = 0.608736
Test set: Balanced classification accuracy = 0.619379
Test set: Disparate impact = 0.563515
Test set: Equal opportunity difference = -0.226970
Test set: Average odds difference = -0.191190
Test set: Theil_index = 0.378022


### 2.2 Random Forest on transformed (DI) dataset

In [55]:
rf_di=RandomForestClassifier(n_estimators=100, verbose=1, n_jobs=30).fit(X_train_rw, y_train_rw,
                                                                            sample_weight=train_pp_bld_rw.instance_weights)
                                                                            
y_rf_pred_di=rf_di.predict(X_test_di)

dataset_debias_test_rf_di = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_rf_di.labels = y_rf_pred_di
display(Markdown("#### Model: RF- debiasing with di - classification metrics"))
classified_metric_debiasing_test_rf_di = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_rf_di,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_rf_di.accuracy())
TPR_rf_di = classified_metric_debiasing_test_rf_di.true_positive_rate()
TNR_rf_di = classified_metric_debiasing_test_rf_di.true_negative_rate()
bal_acc_debiasing_test_rf_di = 0.5*(TPR_rf_di+TNR_rf_di)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_rf_di)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_rf_di.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_rf_di.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_rf_di.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_rf_di.theil_index())

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:   51.4s finished
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:    0.4s finished


#### Model: RF- debiasing with di - classification metrics

Test set: Classification accuracy = 0.753386
Test set: Balanced classification accuracy = 0.616572
Test set: Disparate impact = 0.747360
Test set: Equal opportunity difference = -0.154361
Test set: Average odds difference = -0.178429
Test set: Theil_index = 0.086129


### 2.3 SVM on transformed (DI) dataset

In [56]:
#SVM
svm_di= svm.SVC(kernel='linear', verbose=1, max_iter=10000).fit(X_train_di, y_train_di,
                                                                            sample_weight=train_pp_bld_di.instance_weights)
                                                                            

y_svm_pred_di=svm_di.predict(X_test_di)

[LibSVM].........WARN: libsvm Solver reached max_iter
optimization finished, #iter = 10000
obj = -4030.413983, rho = 0.867157
nSV = 7858, nBSV = 3671
Total nSV = 7858


In [57]:
dataset_debias_test_svm_di = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_svm_di.labels = y_svm_pred_di
display(Markdown("#### Model: SVM- debiasing with DI - classification metrics"))
classified_metric_debiasing_test_svm_di = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_svm_di,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_lr_rw.accuracy())
TPR_svm_di = classified_metric_debiasing_test_svm_di.true_positive_rate()
TNR_svm_di = classified_metric_debiasing_test_svm_di.true_negative_rate()
bal_acc_debiasing_test_svm_di= 0.5*(TPR_lr_di+TNR_lr_di)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_svm_di)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_svm_di.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_svm_di.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_svm_di.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_svm_di.theil_index())

#### Model: SVM- debiasing with DI - classification metrics

Test set: Classification accuracy = 0.708025
Test set: Balanced classification accuracy = 0.619379
Test set: Disparate impact = 1.007426
Test set: Equal opportunity difference = 0.010338
Test set: Average odds difference = 0.005510
Test set: Theil_index = 0.069522


### 2.4 Neural Network on transformed (DI) dataset

In [58]:
#Neural Network

nn_di = Sequential()
nn_di.add(Dense(12, input_dim=129, activation='relu'))
nn_di.add(Dense(8, activation='relu'))
nn_di.add(Dense(1, activation='sigmoid'))

nn_di.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_di.fit(X_train_di, y_train_di, validation_data=(X_test_di, y_test_di), epochs=16, batch_size=1000)
nn_di.evaluate(X_test_di, y_test_di)
y_nn_pred_di=nn_di.predict(X_test_di).round()

Train on 354400 samples, validate on 88600 samples
Epoch 1/16
354400/354400 [==============================] - 2s 5us/sample - loss: 0.6033 - accuracy: 0.6881 - val_loss: 0.5683 - val_accuracy: 0.7132
Epoch 2/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5741 - accuracy: 0.7133 - val_loss: 0.5598 - val_accuracy: 0.7174
Epoch 3/16
354400/354400 [==============================] - 2s 5us/sample - loss: 0.5656 - accuracy: 0.7202 - val_loss: 0.5526 - val_accuracy: 0.7263
Epoch 4/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5572 - accuracy: 0.7274 - val_loss: 0.5453 - val_accuracy: 0.7334
Epoch 5/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5478 - accuracy: 0.7355 - val_loss: 0.5391 - val_accuracy: 0.7386
Epoch 6/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5377 - accuracy: 0.7426 - val_loss: 0.5332 - val_accuracy: 0.7444
Epoch 7/16
354400/354400 [=======================

In [59]:
dataset_debias_test_nn_di = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_nn_di.labels = y_nn_pred_di
display(Markdown("#### Model: NN- debiasing with RW - classification metrics"))
classified_metric_debiasing_test_nn_di = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_nn_di,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_nn_di.accuracy())
TPR_nn_di = classified_metric_debiasing_test_nn_di.true_positive_rate()
TNR_nn_di = classified_metric_debiasing_test_nn_di.true_negative_rate()
bal_acc_debiasing_test_nn_di = 0.5*(TPR_nn_di+TNR_nn_di)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_nn_di)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_nn_di.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_nn_di.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_nn_di.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_nn_di.theil_index())

#### Model: NN- debiasing with RW - classification metrics

Test set: Classification accuracy = 0.748273
Test set: Balanced classification accuracy = 0.636596
Test set: Disparate impact = 0.401421
Test set: Equal opportunity difference = -0.483871
Test set: Average odds difference = -0.528388
Test set: Theil_index = 0.114568


## 3. Learning Fair Representation Pre-processing Algorithm

In [60]:
# Input recontruction quality - Ax
# Fairness constraint - Az
# Output prediction error - Ay
    
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
# Transform training data and align features
train_pp_bld_lfr = TR.fit_transform(train_pp_bld, maxiter=5000, maxfun=5000)

step: 0, loss: 0.6985769994202978, L_x: 0.5090786759673003,  L_y: 0.6388413686980396,  L_z: 0.0044138815627640824
step: 250, loss: 0.6985769558576311, L_x: 0.5090787146424105,  L_y: 0.6388413223650936,  L_z: 0.004413881014148258


KeyboardInterrupt: 

In [ ]:
### Compute fairness metric on transformed dataset-lfr
metric_transf_train_lfr = BinaryLabelDatasetMetric(train_pp_bld_lfr, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset - Learning Fair Representation"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train_lfr.mean_difference())
print("Difference in disparate impact between unprivileged and privileged groups = %f" % metric_transf_train_lfr.disparate_impact())

#### Transformed training dataset - Learning Fair Representation

Difference in mean outcomes between unprivileged and privileged groups = 0.000000
Difference in disparate impact between unprivileged and privileged groups = 1.000000


## Apply classifiers on Transformed Data(Bias Mitigated Dataset via Learning Fair Representation)

In [ ]:
scale_transf_lfr = StandardScaler()
X_train_lfr = scale_transf_lfr.fit_transform(train_pp_bld_lfr.features)
y_train_lfr = train_pp_bld.labels.ravel()

dataset_transf_test_pred_lfr = test_pp_bld.copy(deepcopy=True)
X_test_lfr = scale_transf_lfr.fit_transform(dataset_transf_test_pred_lfr.features)
y_test_lfr = dataset_transf_test_pred_lfr.labels

### 3.1 Logistic Regression on transformed (LFR) dataset

In [ ]:
lmod_lfr = LogisticRegression(max_iter=10000, n_jobs = 30, class_weight='balanced', solver='liblinear')
lmod_lfr.fit(X_train_lfr, y_train_lfr,sample_weight=train_pp_bld_lfr.instance_weights)
y_lr_pred_lfr = lmod_lfr.predict(X_test_lfr)


In [ ]:
dataset_debias_test_lr_lfr = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_lr_lfr.labels = y_lr_pred_lfr.ravel()
display(Markdown("#### Model: LR- debiasing with lfr - classification metrics"))
classified_metric_debiasing_test_lr_lfr = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_lr_lfr,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_lr_lfr.accuracy())
TPR_lr_lfr = classified_metric_debiasing_test_lr_lfr.true_positive_rate()
TNR_lr_lfr = classified_metric_debiasing_test_lr_lfr.true_negative_rate()
bal_acc_debiasing_test_lr_lfr = 0.5*(TPR_lr_lfr+TNR_lr_lfr)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_lr_lfr)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_lr_lfr.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_lr_lfr.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_lr_lfr.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_lr_lfr.theil_index())

#### Model: LR- debiasing with lfr - classification metrics

Test set: Classification accuracy = 0.539977
Test set: Balanced classification accuracy = 0.555877
Test set: Disparate impact = 0.753121
Test set: Equal opportunity difference = -0.088668
Test set: Average odds difference = -0.100114
Test set: Theil_index = 0.462495


### 3.2 Random Forest on transformed (LFR) dataset

In [ ]:
rf_lfr=RandomForestClassifier(n_estimators=100, verbose=1, n_jobs=30).fit(X_train_lfr, y_train_lfr,
                                                                            sample_weight=train_pp_bld_lfr.instance_weights)
                                                                            
y_rf_pred_lfr=rf_lfr.predict(X_test_lfr)

[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 100 out of 100 | elapsed:    0.2s finished


In [ ]:
dataset_debias_test_rf_lfr = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_rf_lfr.labels = y_rf_pred_lfr
display(Markdown("#### Model: RF- debiasing with lfr - classification metrics"))
classified_metric_debiasing_test_rf_lfr = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_rf_lfr,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_rf_lfr.accuracy())
TPR_rf_lfr = classified_metric_debiasing_test_rf_lfr.true_positive_rate()
TNR_rf_lfr = classified_metric_debiasing_test_rf_lfr.true_negative_rate()
bal_acc_debiasing_test_rf_lfr = 0.5*(TPR_rf_lfr+TNR_rf_lfr)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_rf_lfr)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_rf_lfr.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_rf_lfr.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_rf_lfr.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_rf_lfr.theil_index())

#### Model: RF- debiasing with lfr - classification metrics

Test set: Classification accuracy = 0.388973
Test set: Balanced classification accuracy = 0.498901
Test set: Disparate impact = 0.980870
Test set: Equal opportunity difference = -0.007870
Test set: Average odds difference = -0.005535
Test set: Theil_index = 0.822529


### 3.3 SVM classifier on transformed (LFR) dataset

In [ ]:
#SVM
svm_lfr= svm.SVC(kernel='linear', verbose=1, max_iter=10000).fit(X_train_lfr, y_train_lfr,
                                                                            sample_weight=train_pp_bld_lfr.instance_weights)
                                                                            

y_svm_pred_lfr=svm_lfr.predict(X_test_lfr)

[LibSVM].........WARN: libsvm Solver reached max_iter
optimization finished, #iter = 10000
obj = -19991.107642, rho = 0.973414
nSV = 19998, nBSV = 19996
Total nSV = 19998


In [ ]:
dataset_debias_test_svm_lfr = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_svm_lfr.labels = y_svm_pred_di
display(Markdown("#### Model: SVM- debiasing with lfr - classification metrics"))
classified_metric_debiasing_test_svm_lfr = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_svm_lfr,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_lr_lfr.accuracy())
TPR_svm_lfr = classified_metric_debiasing_test_svm_lfr.true_positive_rate()
TNR_svm_lfr = classified_metric_debiasing_test_svm_lfr.true_negative_rate()
bal_acc_debiasing_test_svm_lfr= 0.5*(TPR_lr_lfr+TNR_lr_lfr)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_svm_lfr)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_svm_lfr.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_svm_lfr.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_svm_lfr.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_svm_lfr.theil_index())

#### Model: SVM- debiasing with lfr - classification metrics

Test set: Classification accuracy = 0.539977
Test set: Balanced classification accuracy = 0.555877
Test set: Disparate impact = 1.012777
Test set: Equal opportunity difference = 0.013461
Test set: Average odds difference = 0.010521
Test set: Theil_index = 0.072511


### 3.4 Neural Network on tranformed (LFR) dataset

In [ ]:
#Neural Network

nn_lfr = Sequential()
nn_lfr.add(Dense(12, input_dim=129, activation='relu'))
nn_lfr.add(Dense(8, activation='relu'))
nn_lfr.add(Dense(1, activation='sigmoid'))

nn_lfr.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_lfr.fit(X_train_lfr, y_train_lfr, validation_data=(X_test_lfr, y_test_lfr), epochs=16, batch_size=1000)
nn_lfr.evaluate(X_test_lfr, y_test_lfr)
y_nn_pred_lfr=nn_lfr.predict(X_test_lfr).round()

Train on 354400 samples, validate on 88600 samples
Epoch 1/16
354400/354400 [==============================] - 2s 4us/sample - loss: 0.5872 - accuracy: 0.7109 - val_loss: 1.0264 - val_accuracy: 0.4639
Epoch 2/16
354400/354400 [==============================] - 1s 4us/sample - loss: 0.5688 - accuracy: 0.7245 - val_loss: 1.0162 - val_accuracy: 0.4661
Epoch 3/16
354400/354400 [==============================] - 1s 4us/sample - loss: 0.5687 - accuracy: 0.7247 - val_loss: 1.0086 - val_accuracy: 0.4719
Epoch 4/16
354400/354400 [==============================] - 1s 4us/sample - loss: 0.5688 - accuracy: 0.7247 - val_loss: 1.0048 - val_accuracy: 0.4768
Epoch 5/16
354400/354400 [==============================] - 1s 4us/sample - loss: 0.5687 - accuracy: 0.7247 - val_loss: 0.9875 - val_accuracy: 0.4831
Epoch 6/16
354400/354400 [==============================] - 1s 4us/sample - loss: 0.5686 - accuracy: 0.7247 - val_loss: 0.9843 - val_accuracy: 0.4882
Epoch 7/16
354400/354400 [=======================

In [ ]:
dataset_debias_test_nn_lfr = test_pp_bld.copy(deepcopy = True)
dataset_debias_test_nn_lfr.labels = y_nn_pred_lfr
display(Markdown("#### Model: NN- debiasing with lfr - classification metrics"))
classified_metric_debiasing_test_nn_lfr = ClassificationMetric(test_pp_bld, 
                                                 dataset_debias_test_nn_lfr,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_nn_lfr.accuracy())
TPR_nn_lfr = classified_metric_debiasing_test_nn_lfr.true_positive_rate()
TNR_nn_lfr = classified_metric_debiasing_test_nn_lfr.true_negative_rate()
bal_acc_debiasing_test_nn_lfr = 0.5*(TPR_nn_lfr+TNR_nn_lfr)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_nn_lfr)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_nn_lfr.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_nn_lfr.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_nn_lfr.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_nn_lfr.theil_index())

#### Model: NN- debiasing with lfr - classification metrics

Test set: Classification accuracy = 0.530169
Test set: Balanced classification accuracy = 0.505680
Test set: Disparate impact = 1.010694
Test set: Equal opportunity difference = 0.043368
Test set: Average odds difference = 0.008013
Test set: Theil_index = 0.418910


# 4. Optim Pre-processing Algorithm

Optimized preprocessing is a preprocessing technique that learns a probabilistic transformation that edits the features and labels in the data with group fairness, individual distortion, and data fidelity constraints and objective

In [ ]:
# optim_options = {
#             "distortion_fun": get_distortion_german,
#             "epsilon": 0.05,
#             # "clist": [0.99],
#             "dlist": [.1]
#         }
# OP = OptimPreproc(OptTools, optim_options)

# OP = OP.fit(train_pp_bld)
# train_pp_bld_op = OP.transform(train_pp_bld, transform_Y=True)

# train_pp_bld_op = train_pp_bld.align_datasets(train_pp_bld_op)

# Result (Pre-Processing Algorithms)

In [ ]:
# print('Accuracy on Test Set [Preprocessing Algorithm:]:')

# result_acc = pd.DataFrame({'Logistic Regression': [acc_lr_biased, 
#                                                 acc_lr_rw,
#                                                 acc_lr_di,
#                                                 acc_lr_lfr],
#                         'Random Forest':        [acc_rf_biased,
#                                                 acc_rf_rw,
#                                                 acc_rf_di,
#                                                 acc_rf_lfr],
#                         'SVM':                  [acc_svm_biased,
#                                                 acc_svm_rw,
#                                                 acc_svm_di,
#                                                 acc_svm_lfr],
#                         'Neural Network':       [acc_nn_biased,
#                                                 acc_nn_rw,
#                                                 acc_nn_di,
#                                                 acc_nn_lfr]
#                         }, 
#                      index =['Biased', 'Reweighing', 'Disparate Impact Remover', 'Learning Fair Representation'])
# result_acc

In [ ]:
# Creating data
Information_RW = {'Classifier': ["Logistic Regression", "", "","", "", "", 
                            "Random Forest", "", "", "", "", "",
                            "SVM", "", "", "", "", "", 
                            "Neural Network", "", "", "", "", ""],
                
               'Metrics': ["Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index"
                        ],
                
               'Score': [classified_metric_debiasing_test_lr_rw.accuracy(), bal_acc_debiasing_test_lr_rw, classified_metric_debiasing_test_lr_rw.disparate_impact(), classified_metric_debiasing_test_lr_rw.equal_opportunity_difference(), classified_metric_debiasing_test_lr_rw.average_odds_difference(), classified_metric_debiasing_test_lr_rw.theil_index(), 
                        classified_metric_debiasing_test_rf_rw.accuracy(), bal_acc_debiasing_test_rf_rw, classified_metric_debiasing_test_rf_rw.disparate_impact(), classified_metric_debiasing_test_rf_rw.equal_opportunity_difference(), classified_metric_debiasing_test_rf_rw.average_odds_difference(), classified_metric_debiasing_test_rf_rw.theil_index(), 
                        classified_metric_debiasing_test_svm_rw.accuracy(), bal_acc_debiasing_test_svm_rw, classified_metric_debiasing_test_svm_rw.disparate_impact(), classified_metric_debiasing_test_svm_rw.equal_opportunity_difference(), classified_metric_debiasing_test_svm_rw.average_odds_difference(), classified_metric_debiasing_test_svm_rw.theil_index(),  
                        classified_metric_debiasing_test_nn_rw.accuracy(), bal_acc_debiasing_test_nn_rw, classified_metric_debiasing_test_nn_rw.disparate_impact(), classified_metric_debiasing_test_nn_rw.equal_opportunity_difference(), classified_metric_debiasing_test_nn_rw.average_odds_difference(), classified_metric_debiasing_test_rf_rw.theil_index()
                        ]}
 
# Dataframing the whole data
df_RW = pd.DataFrame(Information_RW)

In [ ]:
# print('Comparisons based on DI Preprocessing Algorithm:]:')
# Creating data
Information_DI = {'Classifier': ["Logistic Regression", "", "","", "", "", 
                            "Random Forest", "", "", "", "", "",
                            "SVM", "", "", "", "", "", 
                            "Neural Network", "", "", "", "", ""],
                
               'Metrics': ["Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index"
                        ],
                
               'Score': [classified_metric_debiasing_test_lr_di.accuracy(), bal_acc_debiasing_test_lr_di, classified_metric_debiasing_test_lr_di.disparate_impact(), classified_metric_debiasing_test_lr_di.equal_opportunity_difference(), classified_metric_debiasing_test_lr_di.average_odds_difference(), classified_metric_debiasing_test_lr_di.theil_index(), 
                        classified_metric_debiasing_test_rf_di.accuracy(), bal_acc_debiasing_test_rf_di, classified_metric_debiasing_test_rf_di.disparate_impact(), classified_metric_debiasing_test_rf_di.equal_opportunity_difference(), classified_metric_debiasing_test_rf_di.average_odds_difference(), classified_metric_debiasing_test_rf_di.theil_index(), 
                        classified_metric_debiasing_test_svm_di.accuracy(), bal_acc_debiasing_test_svm_di, classified_metric_debiasing_test_svm_di.disparate_impact(), classified_metric_debiasing_test_svm_di.equal_opportunity_difference(), classified_metric_debiasing_test_svm_di.average_odds_difference(), classified_metric_debiasing_test_svm_di.theil_index(),  
                        classified_metric_debiasing_test_nn_di.accuracy(), bal_acc_debiasing_test_nn_di, classified_metric_debiasing_test_nn_di.disparate_impact(), classified_metric_debiasing_test_nn_di.equal_opportunity_difference(), classified_metric_debiasing_test_nn_di.average_odds_difference(), classified_metric_debiasing_test_rf_di.theil_index()
                        ]}
 
# Dataframing the whole data
df_DI = pd.DataFrame(Information_DI)
 
# # Showing the above data
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(df_DI)

In [ ]:
# print('Comparisons based on Reweighing Preprocessing Algorithm:]:')
# Creating data
Information_lfr = {'Classifier': ["Logistic Regression", "", "","", "", "", 
                            "Random Forest", "", "", "", "", "",
                            "SVM", "", "", "", "", "", 
                            "Neural Network", "", "", "", "", ""],
                
               'Metrics': ["Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index",
                        "Classification accuracy", " Balanced classification accuracy", "Disparate impact", "Equal opportunity difference", "Average odds difference", "Theil_index"
                        ],
                
               'Score': [classified_metric_debiasing_test_lr_lfr.accuracy(), bal_acc_debiasing_test_lr_lfr, classified_metric_debiasing_test_lr_lfr.disparate_impact(), classified_metric_debiasing_test_lr_lfr.equal_opportunity_difference(), classified_metric_debiasing_test_lr_lfr.average_odds_difference(), classified_metric_debiasing_test_lr_lfr.theil_index(), 
                        classified_metric_debiasing_test_rf_lfr.accuracy(), bal_acc_debiasing_test_rf_lfr, classified_metric_debiasing_test_rf_lfr.disparate_impact(), classified_metric_debiasing_test_rf_lfr.equal_opportunity_difference(), classified_metric_debiasing_test_rf_lfr.average_odds_difference(), classified_metric_debiasing_test_rf_lfr.theil_index(), 
                        classified_metric_debiasing_test_svm_lfr.accuracy(), bal_acc_debiasing_test_svm_lfr, classified_metric_debiasing_test_svm_lfr.disparate_impact(), classified_metric_debiasing_test_svm_lfr.equal_opportunity_difference(), classified_metric_debiasing_test_svm_lfr.average_odds_difference(), classified_metric_debiasing_test_svm_lfr.theil_index(),  
                        classified_metric_debiasing_test_nn_lfr.accuracy(), bal_acc_debiasing_test_nn_lfr, classified_metric_debiasing_test_nn_lfr.disparate_impact(), classified_metric_debiasing_test_nn_lfr.equal_opportunity_difference(), classified_metric_debiasing_test_nn_lfr.average_odds_difference(), classified_metric_debiasing_test_rf_lfr.theil_index()
                        ]}
 
# Dataframing the whole data
df_LFR = pd.DataFrame(Information_lfr)


In [ ]:
# Showing the above data
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print('Comparisons based on Reweighing Preprocessing Algorithm:]:\n')
    print(df_RW)
    print ('\n')
    print('Comparisons based on Disparate Impact Preprocessing Algorithm:]:\n')
    print(df_DI)
    print ('\n')
    print('Comparisons based on Learning Fair Representation Preprocessing Algorithm:]:\n')
    print(df_LFR)

Comparisons based on Reweighing Preprocessing Algorithm:]:

             Classifier                            Metrics     Score
0   Logistic Regression            Classification accuracy  0.707280
1                         Balanced classification accuracy  0.524883
2                                         Disparate impact  0.971971
3                             Equal opportunity difference -0.006803
4                                  Average odds difference -0.013391
5                                              Theil_index  0.078282
6         Random Forest            Classification accuracy  0.753894
7                         Balanced classification accuracy  0.616837
8                                         Disparate impact  0.747663
9                             Equal opportunity difference -0.151578
10                                 Average odds difference -0.176421
11                                             Theil_index  0.086497
12                  SVM            Classifi

### In-processing Algorithms: 

With in-processing techniques, we want to create a classifier that is explicitly aware of our fairness goals. That is, in training the classifier, it is not enough to simply optimize for accuracy on the training data. Instead, we modify the loss function to account simultaneously for our two goals: our model should be both accurate and fair.

1. Adversial Debiasing In-processing algorithm
2. Meta Fair Classifier
3. Prejudice Remover
4. Exponentiated Gradient Reduction


### 1. Adversial Debiasing In-processing Algorithm
Adversarial debiasing is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary’s ability to determine the protected attribute from the predictions [5]. This approach leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit.

### Learn plane model without debiasing


In [ ]:
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

2022-06-13 11:36:29.427514: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-06-13 11:36:29.427565: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
plain_model.fit(train_pp_bld)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2022-06-13 11:36:34.173478: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


epoch 0; iter: 0; batch classifier loss: 0.677886
epoch 0; iter: 200; batch classifier loss: 0.555166
epoch 0; iter: 400; batch classifier loss: 0.577297
epoch 0; iter: 600; batch classifier loss: 0.632255
epoch 0; iter: 800; batch classifier loss: 0.633876
epoch 0; iter: 1000; batch classifier loss: 0.569676
epoch 0; iter: 1200; batch classifier loss: 0.584164
epoch 0; iter: 1400; batch classifier loss: 0.558872
epoch 0; iter: 1600; batch classifier loss: 0.536281
epoch 0; iter: 1800; batch classifier loss: 0.525641
epoch 0; iter: 2000; batch classifier loss: 0.590979
epoch 0; iter: 2200; batch classifier loss: 0.562403
epoch 0; iter: 2400; batch classifier loss: 0.548421
epoch 0; iter: 2600; batch classifier loss: 0.482196
epoch 1; iter: 0; batch classifier loss: 0.516783
epoch 1; iter: 200; batch classifier loss: 0.518328
epoch 1; iter: 400; batch classifier loss: 0.564152
epoch 1; iter: 600; batch classifier loss: 0.583995
epoch 1; iter: 800; batch classifier loss: 0.506066
epoch 1

In [ ]:
# Apply the plain model to test data
dataset_nodebiasing_train_ad = plain_model.predict(train_pp_bld)
dataset_nodebiasing_test_ad = plain_model.predict(test_pp_bld)

### Apply Adversial Debiasing pre-processing algorithm

In [ ]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [ ]:
# Learn parameters with debias set to True
debiased_model_ad = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [ ]:
debiased_model_ad.fit(train_pp_bld)

epoch 0; iter: 0; batch classifier loss: 0.687576; batch adversarial loss: 0.726831
epoch 0; iter: 200; batch classifier loss: 0.668309; batch adversarial loss: 0.748306
epoch 0; iter: 400; batch classifier loss: 0.602222; batch adversarial loss: 0.592351
epoch 0; iter: 600; batch classifier loss: 0.541700; batch adversarial loss: 0.477641
epoch 0; iter: 800; batch classifier loss: 0.608029; batch adversarial loss: 0.453457
epoch 0; iter: 1000; batch classifier loss: 0.626866; batch adversarial loss: 0.477429
epoch 0; iter: 1200; batch classifier loss: 0.587422; batch adversarial loss: 0.410003
epoch 0; iter: 1400; batch classifier loss: 0.907972; batch adversarial loss: 0.573736
epoch 0; iter: 1600; batch classifier loss: 0.912307; batch adversarial loss: 0.586690
epoch 0; iter: 1800; batch classifier loss: 0.665068; batch adversarial loss: 0.449919
epoch 0; iter: 2000; batch classifier loss: 0.579663; batch adversarial loss: 0.465407
epoch 0; iter: 2200; batch classifier loss: 0.5623

In [ ]:
# Apply the adversial model to test data
dataset_debiasing_train_ad = debiased_model_ad.predict(train_pp_bld)
dataset_debiasing_test_ad = debiased_model_ad.predict(test_pp_bld)

### Metrics comparisons before and after adversial debiasing

In [ ]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train_ad = BinaryLabelDatasetMetric(dataset_debiasing_train_ad, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train_ad.mean_difference())

metric_dataset_debiasing_test_ad = BinaryLabelDatasetMetric(dataset_debiasing_test_ad, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test_ad.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test_ad = ClassificationMetric(test_pp_bld, 
                                                 dataset_nodebiasing_test_ad,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test_ad.accuracy())
TPR_ad = classified_metric_nodebiasing_test_ad.true_positive_rate()
TNR_ad = classified_metric_nodebiasing_test_ad.true_negative_rate()
bal_acc_nodebiasing_test_ad = 0.5*(TPR_ad+TNR_ad)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test_ad)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test_ad.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test_ad.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test_ad.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test_ad.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test_ad = ClassificationMetric(test_pp_bld, 
                                                 dataset_debiasing_test_ad,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test_ad.accuracy())
TPR_dad = classified_metric_debiasing_test_ad.true_positive_rate()
TNR_dad = classified_metric_debiasing_test_ad.true_negative_rate()
bal_acc_debiasing_test_dad = 0.5*(TPR_dad+TNR_dad)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_dad)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_ad.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_ad.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_ad.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_ad.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.322358
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.322358


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.540159
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.539525


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.742596
Test set: Balanced classification accuracy = 0.617089
Test set: Disparate impact = 0.361564
Test set: Equal opportunity difference = -0.518308
Test set: Average odds difference = -0.612265
Test set: Theil_index = 0.104408


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.698420
Test set: Balanced classification accuracy = 0.604537
Test set: Disparate impact = 0.391340
Test set: Equal opportunity difference = -0.465564
Test set: Average odds difference = -0.518350
Test set: Theil_index = 0.173040


Comment 2: Based on the metrics score for dataset and classification, we can say that adversial debiasing will not reduce biasness.

## 2. Meta Fair Classifier

### Algorithm without debiasing 

In [ ]:
# biased_model = MetaFairClassifier(tau=0, sensitive_attr="age", type="fdr").fit(train_pp_bld)

In [ ]:
# dataset_bias_test_mf = biased_model.predict(test_pp_bld)


In [ ]:
# display(Markdown("#### Model - without Meta Fair debiasing - classification metrics"))
# classified_metric_bias_test_mf = ClassificationMetric(test_pp_bld, dataset_bias_test_mf,
#                                                    unprivileged_groups=unprivileged_groups,
#                                                    privileged_groups=privileged_groups)
# print("Test set: Classification accuracy = {:.3f}".format(classified_metric_bias_test_mf.accuracy()))
# TPR_mf = classified_metric_bias_test_mf.true_positive_rate()
# TNR_mf = classified_metric_bias_test_mf.true_negative_rate()
# bal_acc_bias_test_mf = 0.5*(TPR_mf+TNR_mf)
# print("Test set: Balanced classification accuracy = {:.3f}".format(bal_acc_bias_test_mf))
# print("Test set: Disparate impact = {:.3f}".format(classified_metric_bias_test_mf.disparate_impact()))
# fdr_mf = classified_metric_bias_test_mf.false_discovery_rate_ratio()
# fdr_mf = min(fdr_mf, 1/fdr_mf)
# print("Test set: False discovery rate ratio = {:.3f}".format(fdr_mf))

### Learn debiased meta classifier

In [ ]:
# debiased_model_mf = MetaFairClassifier(tau = 0.7, sensitive_attr="age", type="fdr").fit(train_pp_bld)

## 3. Prejudice Remover
Prejudice remover is an in-processing technique that adds a discrimination-aware regularization term to the learning objective

In [ ]:
# scale_transf_pr = StandardScaler()
# train_pp_bld_sc = scale_transf_pr.fit_transform(train_pp_bld)
model_pr = PrejudiceRemover(sensitive_attr= 'age', eta=25.0)
pr_orig_scaler = StandardScaler()

dataset = train_pp_bld.copy()
dataset.features = pr_orig_scaler.fit_transform(dataset.features)

debiased_model_pr = model_pr.fit(dataset)

/data2/users/ssultana/anaconda3/envs/aif360/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/data2/users/ssultana/anaconda3/envs/aif360/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

In [ ]:
## Validating PR model
dataset_debias_test_pr = debiased_model_pr.predict(test_pp_bld)

In [ ]:
classified_metric_debiasing_test_pr = ClassificationMetric(test_pp_bld, dataset_debias_test_pr,
                                                   unprivileged_groups=unprivileged_groups,
                                                   privileged_groups=privileged_groups)
print("Test set: Classification accuracy = {:.3f}".format(classified_metric_debiasing_test_pr.accuracy()))
TPR_pr = classified_metric_debiasing_test_pr.true_positive_rate()
TNR_pr = classified_metric_debiasing_test_pr.true_negative_rate()
bal_acc_debiasing_test_pr = 0.5*(TPR_pr+TNR_pr)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test_pr)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test_pr.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test_pr.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test_pr.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test_pr.theil_index())

Test set: Classification accuracy = 0.725
Test set: Balanced classification accuracy = 0.627701
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -1.000000
Test set: Average odds difference = -1.000000
Test set: Theil_index = 0.147542


## 4. Exponentiated Gradient Reduction

Exponentiated gradient reduction is an in-processing technique that reduces fair classification to a sequence of cost-sensitive classification problems, returning a randomized classifier with the lowest empirical error subject to fair classification constraints.

In [ ]:
estimator = LogisticRegression(solver='lbfgs')

In [ ]:
np.random.seed(0) #need for reproducibility
exp_grad_red = ExponentiatedGradientReduction(estimator=estimator, 
                                              constraints="EqualizedOdds",
                                              drop_prot_attr=False)
exp_grad_red.fit(train_pp_bld)
exp_grad_red_pred = exp_grad_red.predict(test_pp_bld)

In [ ]:
metric_test_egr = ClassificationMetric(test_pp_bld, 
                                    exp_grad_red_pred,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

display(Markdown("#### Accuracy for Exponentiated Gradient Reduction"))
egr_acc = metric_test_egr.accuracy()
print(egr_acc)

#### Accuracy for Exponentiated Gradient Reduction

0.7040293453724605


# Post-Processing Algorithm

## 1. Odds equalizing post-processing algorithm

In [ ]:
dataset_orig_train_pred = train_pp_bld.copy(deepcopy=True)
# cost constraint of fnr will optimize generalized false negative rates, that of
# fpr will optimize generalized false positive rates, and weighted will optimize
# a weighted combination of both
cost_constraint = "fnr" # "fnr", "fpr", "weighted"
#random seed for calibrated equal odds prediction
randseed = 12345679 

In [ ]:

# Learn parameters to equalize odds and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint=cost_constraint,
                                     seed=randseed)
cpp = cpp.fit(train_pp_bld, dataset_orig_train_pred)

In [ ]:
#Transform test data using the post processing algorithm
dataset_orig_test_pred = test_pp_bld.copy(deepcopy=True)
dataset_transf_test_pred = cpp.predict(dataset_orig_test_pred)

In [ ]:
""" GFP: Return the generalized number of false positives, :math:`GFP`, the
        weighted sum of predicted scores where true labels are 'favorable',
        optionally conditioned on protected attributes."""
cm_transf_test = ClassificationMetric(test_pp_bld, dataset_transf_test_pred,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed testing dataset"))
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_transf_test.difference(cm_transf_test.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_transf_test.difference(cm_transf_test.generalized_false_negative_rate))
print("Test set: Classification accuracy of Calibrated Equality of Odds = {:.3f}".format(cm_transf_test.accuracy()))

#### Transformed testing dataset

Difference in GFPR between unprivileged and privileged groups
0.0
Difference in GFNR between unprivileged and privileged groups
0.0
Test set: Classification accuracy of Calibrated Equality of Odds = 0.992


## 2. Reject Option Classification: 
Reject option classification is a postprocessing technique that gives favorable outcomes to unpriviliged groups and unfavorable outcomes to priviliged groups in a confidence band around the decision boundary with the highest uncertainty

In [ ]:
# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [ ]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(train_pp_bld, dataset_orig_train_pred)

In [ ]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.0100
Optimal ROC margin = 0.0000
